In [9]:
import numpy as np
from torchvision.datasets import MNIST

def download_mnist(is_train: bool):
    dataset = MNIST(root='./data',
                    transform=lambda x: np.array(x).flatten(), # we convert 28 x 28 into 784 x 1, same as reshaping
                    download=True,
                    train=is_train)

    mnist_data = []
    mnist_labels = []
    for image, label in dataset:
        mnist_data.append(image)
        mnist_labels.append(label)

    return mnist_data, mnist_labels


train_X, train_y = download_mnist(True) # download the train batches
test_X, test_y = download_mnist(False) # download the test batches

In [10]:
# normalize the pixels values from [0, 255] to [0, 1]
train_X = np.array(train_X) / 255.0
train_y = np.array(train_y)
test_X = np.array(test_X) / 255.0
test_y = np.array(test_y)
print(train_X.shape)
print(train_y.shape)

(60000, 784)
(60000,)


In [11]:
class MLP:
    def __init__(self, input_size, hidden_size, output_size, l2_lambda = 0.001):
        # initialize weights (as random values of a normal distribution), biases (all 0, initially) for input state -> hidden state, and hidden state -> output

        # input > hidden_state
        self.W1 = np.random.randn(input_size, hidden_size) * 0.01 # has this size because we need an array of size 784 (input_size) weights for each 100 (hidden_size) layers
        self.b1 = np.zeros((1, hidden_size)) # bias vector for all hidden layer in total

        # hidden_state > output
        self.W2 = np.random.randn(hidden_size, output_size) * 0.01 # same as before but we turn 100 hidden state results into 10 outputs
        self.b2 = np.zeros((1, output_size)) # bias vector for all output
        self.l2_lambda = l2_lambda # constant used for L2 regularization

    # defining math functions, using reLU as an activation function:
    # we use this function to introduce non-linearity, since linear transformations alone cannot comprehend the relations between complex data
    # another useful trait is that they help avoid "vanishing gradients", aka, cases where the gradients get so small it is difficult for the values of the problem to approach
    # the desired values and weights and such get updated slower and slower
    def relu(self, x):
        return np.maximum(0, x)
    
    def relu_derivative(self, x):
        return (x > 0).astype(float) # cuz if x > 0, reLU(x) = x so derivative is 1, else it is derivative of constant 0 = 0
    
    # we use softmax to convert values into probabilities
    def softmax(self, x):
        # we use axis=1 to calculate these values across a particular sample/row (a size 784 vector x), otherwise it's nonsensical to do it on columns
        # we use keepdims=True so we get as a result an array of compatible size for other array-wide operations
        exp_x = np.exp(x - np.max(x, axis=1, keepdims=True)) # array-wide (batch) operations, faster than doing a "for" loop
        return exp_x / np.sum(exp_x, axis=1, keepdims=True)


ReLU definition:
$$
ReLU(x) = max(0, x)
$$

Derivative of ReLU:
$$
\frac{d}{dx}ReLU(x) = 
\begin{cases} 
1 & \text{if } x > 0, \\
0 & \text{if } x \leq 0, \\
\end{cases}
$$

Softmax definition:
$$
\text{Softmax}(x_i) = \frac{e^{x_i}}{\sum_{j} e^{x_j}}
$$

for a value $x_i$ - in our case, $x_i$ is equal to the difference of the $i$-th element of the vector (batch) $X$ and the maximum value from this vector; this is done to prevent overflow and stabilize the data by reducing numbers across the board.

In [12]:
class MLP(MLP):  # extending the previous class instead of continuing it in the same cell for the sake of readable code and markdown cells inbetween
    def forward(self, X):
        # forward pass: data flowing from input -> hidden
        self.z1 = X.dot(self.W1) + self.b1 # taking inputs and connecting them into hidden layer pre-activation vector z
        self.a1 = self.relu(self.z1) # applying activation of the layer, in our case, reLU
        
        # forward pass: data flowing from hidden -> output
        self.z2 = self.a1.dot(self.W2) + self.b2 # taking hidden layer results and connecting them into outputs
        output = self.softmax(self.z2) # converting raw numbers into probabilities, aka the output classification values
         
        return output

Forward propagation:

Our values go through all the layers to reach the end as output, using the following formulas:
$$
z_i = W_i \cdot x + b_i
$$
for specific values. In our code, we do batch operations, meaning that this is actually applied to the entire vector; as such, we more accurately do
$$
Z = X \cdot W + B
$$
since $X$ has size (N, 784) (for N the amount of samples we are running) and $W$ as a weight matrix has size (784, 100), multiplication can only be done that way.

In [13]:
class MLP(MLP): 
    def compute_loss(self, y, output):
        # convert to one-hot encoding matrix
        # the way this works is basically ensuring each array in the 10 x 10 matrix created defines uniquely a value of y
        # for example, if y[3] = 3, then the corresponding row in the matrix will be [0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
        y_one_hot = np.eye(10)[y]

        # cross-entropy loss (average over an entire batch, because we divide by y.shape[0])
        # measures the difference between the predicted class probabilities (output) and the true class labels (y_one_hot)
        cross_entropy_loss = -np.sum(y_one_hot * np.log(output + 1e-8)) / y.shape[0] # usual entropy formula averaged, with 1e-8 added to prevent log 0 from appearing
        
        # L2 regularization loss
        # we use L2 regularization to prevent overfitting, adding a penalty which increases with bigger weights
        # factor of 1/2 is included to simplify the derivative when calculating gradients, apparently good practice
        # basically calculated like in the formual below
        l2_loss = (self.l2_lambda / 2) * (np.sum(np.square(self.W1)) + np.sum(np.square(self.W2)))
        
        # total loss is the cross-entropy loss + L2 regularization loss
        # loss measures the difference between the expected results and the obtained results, in general
        total_loss = cross_entropy_loss + l2_loss
        return total_loss

Cross entropy formula:
$$
\text{CrossEntropy}(Y_{\text{one-hot}}, Y_{\text{output}}) = - \sum y_{\text{one-hot}} \log_2 ( y_{\text{output}} )
$$

L2 regularization loss:
$$
\text{Loss}_{\text{L2}} = \frac{\lambda}{2} \left( \sum_{i} W_{1,i}^2 + \sum_{j} W_{2,j}^2 \right)
$$

where:
- $W_1$ and $W_2$ are the weight matrices,
- $\lambda$ is the regularization parameter ('l2_lambda' in code).

Total loss:
$$
\text{Total Loss} = \frac{\text{CrossEntropy}(Y_{\text{one-hot}}, Y_{\text{output}})}{N} + \text{Loss}_{\text{L2}}
$$

where $N$ is the batch size.

In [14]:
class MLP(MLP):
    def backward(self, X, y, output, learning_rate):
        # whereas forwardpropagation is used to go from input through hidden layers until the output, to properly train the network, we go BACK and readjust values as needed
        # to approach more accurate predictions - this is the basis of backward propagation
        # when we say gradient, we refer to the change that must be applied to the parameters of the network in order for higher and higher accuracy

        # one-hot encode the labels
        y_one_hot = np.eye(10)[y]
        
        # GRADIENT FOR OUTPUT LAYER IN TERMS OF OBTAINED RESULTS
        # difference between predicted values and true values - basically derivative of loss function, can be considered equivalent to an error function
        d_z2 = output - y_one_hot

        # then we calculate gradient of the weights
        # we transpose the activation value array because it has size (N, 100), and we want (100, N) to be able to multiply it with the output layer gradient array of size (N, 10)
        d_W2 = self.a1.T.dot(d_z2) / X.shape[0] + self.l2_lambda * self.W2 # gradient of L2 regularization term, formula below
        d_b2 = np.sum(d_z2, axis=0, keepdims=True) / X.shape[0] # gradient of biases, calculated by summing the errors across the samples and averaging over the batch
        
        # GRADIENT FOR HIDDEN LAYER, GOING BACK, USING THE CHAIN RULE (where necessary)
        d_a1 = d_z2.dot(self.W2.T) # adjust activation array gradients for the previous layer in terms of the output and weights of the current

        # gradient of the loss with respect to pre-activation vector z_1
        # since the hidden layer uses ReLU as its activation, we multiply d_a1 element-wise (batch operations, as always) by relu_derivative(self.z1) to apply the chain rule
        # this gives the gradient with respect to z1, where ReLU only allows positive values to pass through
        d_z1 = d_a1 * self.relu_derivative(self.z1)
        
        # similar to before, we use the chain rule this time:
        # this calculates the gradient for W1 using the input data X and the backpropagated gradient d_z1, dividing by batch size to normalize and adding L2 regularization term
        d_W1 = X.T.dot(d_z1) / X.shape[0] + self.l2_lambda * self.W1
        d_b1 = np.sum(d_z1, axis=0, keepdims=True) / X.shape[0] # sums the gradient across all examples in the batch for each neuron, normalized by dividing by batch size
        
        # and finally, we update weights and biases (entire point of backpropagation)
        # these adjustments minimize the loss by a small step in the direction opposite to the gradient, moving closer to the loss function’s minimum, like in the graph
        self.W1 -= learning_rate * d_W1
        self.b1 -= learning_rate * d_b1
        self.W2 -= learning_rate * d_W2
        self.b2 -= learning_rate * d_b2


***Calculating gradients of the output:***

Gradient of the loss (equivalent to an error function):
$$
\mathrm{d}Z_2 = \hat{Y} - Y = Y_{\text{output}} - Y_{\text{one-hot}}
$$

Gradient of the weights:
$$
\mathrm{d}W_2 = \frac{a_1^T \cdot \mathrm{d}Z_2}{N} + \lambda W_2
$$

The value added (compared to default backpropagation) is the gradient of the L2 regularization term with respect to the weights:
$$
\frac{\partial}{\partial W_k} \left( \frac{\lambda}{2} \sum_{i} W_{k,i}^2 \right) = \lambda W_k, \forall k \in \{1, 2\}
$$

Gradient of the biases:
$$
\mathrm{d}b_2 = \frac{1}{N} \sum \mathrm{d}Z_2
$$

***Going back, updating the gradients of the previous (hidden) layer in terms of the output:***

Gradient of the activation vector:
$$
\mathrm{d}a_1 = \frac{\partial L}{\partial a_1} = \frac{\partial L}{\partial Z_2} \cdot \frac{\partial Z_2}{\partial a_1} = \mathrm{d}Z_2 \cdot W_2^T
$$

Gradient of the loss:
$$
\mathrm{d}Z_1 = \frac{\partial L}{\partial Z_1} = \frac{\partial L}{\partial a_1} \cdot \frac{\partial a_1}{\partial Z_1} = \mathrm{d}a_1 \cdot \frac{\mathrm{d}}{\mathrm{d}Z_1}\text{ReLU}(Z_1)
$$

Gradient of the weights:
$$
\mathrm{d}W_1 = \frac{\partial L}{\partial W_1} = \frac{\partial L}{\partial a_1} \cdot \frac{\partial a_1}{\partial Z_1} \cdot \frac{\partial Z_1}{\partial W_1} + \lambda \cdot W_1 = \frac{1}{N} X^T \cdot \mathrm{d}Z_1 + \lambda \cdot W_1
$$

$$
\mathrm{d}b_1 = \frac{\partial L}{\partial b_1} = \frac{1}{N} \sum \mathrm{d}Z_1
$$

In [15]:
def accuracy(predictions, labels):
    return np.mean(predictions == labels) * 100

def train_mlp(mlp, train_X, train_y, test_X, test_y, epochs, batch_size=64, learning_rate=0.01):
    for epoch in range(epochs):
        # batch gradient descent
        for i in range(0, len(train_X), batch_size):
            # this range is equivalent to for(i = 0; i <= len(train_X); i += 64) in C++

            X_batch = train_X[i:i+batch_size] # take a batch
            y_batch = train_y[i:i+batch_size] # take a batch
            
            # forwardpropagation:
            output = mlp.forward(X_batch)
            
            # backpropagation:
            mlp.backward(X_batch, y_batch, output, learning_rate)
        
        # training accuracy:
        train_predictions = np.argmax(mlp.forward(train_X), axis=1) # array containing labels with highest probability chosen for current batch of values
        train_accuracy = accuracy(train_predictions, train_y) # comparing predictions with actual labels for current batch

        # testing accuracy
        test_predictions = np.argmax(mlp.forward(test_X), axis=1)
        test_accuracy = accuracy(test_predictions, test_y)
        loss = mlp.compute_loss(y_batch, output) # compute loss for current batch
        
        if (epoch + 1) % 5 == 0:
            print(f"Epoch {epoch+1}/{epochs} - Training Accuracy: {train_accuracy:.5f}% - Validation Accuracy: {test_accuracy:.5f}%, Loss: {loss:.4f}")

# Initialize and train the MLP
mlp = MLP(input_size=784, hidden_size=100, output_size=10, l2_lambda=0.001)
train_mlp(mlp, train_X, train_y, test_X, test_y, epochs=50, batch_size=64, learning_rate=0.01)


Epoch 5/50 - Training Accuracy: 90.13667% - Validation Accuracy: 90.55000%, Loss: 0.2165
Epoch 10/50 - Training Accuracy: 91.85333% - Validation Accuracy: 92.13000%, Loss: 0.1604
Epoch 15/50 - Training Accuracy: 92.98167% - Validation Accuracy: 93.04000%, Loss: 0.1403
Epoch 20/50 - Training Accuracy: 93.93833% - Validation Accuracy: 93.83000%, Loss: 0.1300
Epoch 25/50 - Training Accuracy: 94.62000% - Validation Accuracy: 94.52000%, Loss: 0.1225
Epoch 30/50 - Training Accuracy: 95.20000% - Validation Accuracy: 94.98000%, Loss: 0.1161
Epoch 35/50 - Training Accuracy: 95.58000% - Validation Accuracy: 95.31000%, Loss: 0.1121
Epoch 40/50 - Training Accuracy: 95.90000% - Validation Accuracy: 95.61000%, Loss: 0.1089
Epoch 45/50 - Training Accuracy: 96.21167% - Validation Accuracy: 95.88000%, Loss: 0.1066
Epoch 50/50 - Training Accuracy: 96.46667% - Validation Accuracy: 96.22000%, Loss: 0.1049
